## imports

In [66]:
import re
import json

alphanumeric='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
dot =  '('+('|'.join(alphanumeric))+')'

# print(dot)


# Regex to Postfix

In [83]:
def validate_regex(regex):
    try:
       re.compile(regex)
       return True
    except:
       return False 
    



def handle_dot(regex):
   return re.sub(r'\.', dot, regex)

def handle_range(regex):
    new_regex = ""
    replace = False
    start = None
    end= None
    for i in range(len(regex)):
        if regex[i] == '[':
            new_regex += '('
            replace =True
        elif regex[i]== ']':
             new_regex += ')'
             replace =False
        elif regex[i] == '-':
            replace = True
            start = alphanumeric.index(regex[i-1])
            end = alphanumeric.index(regex[i+1])
            for j in range(start+1,end):
                new_regex += alphanumeric[j]
                if(replace and regex[i+1] != ']'):
                    new_regex+='|'
        else:
           new_regex += regex[i] 
           if(replace and regex[i+1] != ']'):
                    new_regex+='|'


    return new_regex  


def concatenate(regex):
    new_regex = ""
    for i in range(len(regex)-1):
        new_regex += regex[i] 
        if regex[i] in ['*','+','?',')',']'] and regex[i+1] not in ['*','+','?',')',']','.', '|']:
            new_regex+="."
        elif regex[i] in alphanumeric and  (regex[i+1] in alphanumeric or regex[i+1] in ['(','[']):
            new_regex+="."
    
    new_regex += regex[-1]

    return new_regex

def infix_to_postfix(regex):
    queue = []
    stack = []
    operators={'(':0,"|":1, ".":2, "?":3, "+":4, "*":5}

    for i in range(len(regex)):
        if regex[i] == '(':
            stack.append(regex[i])
        elif regex[i] == ')':
            while stack[-1] != "(":  
                queue.append(stack.pop()) 
            stack.pop()     
        elif regex[i] in operators:
            while stack  and operators[regex[i]] <= operators[stack[-1]]:
                queue.append(stack.pop())
            stack.append(regex[i])

        else :
            queue.append(regex[i])    
    
    while stack:
        queue.append(stack.pop())

    return ''.join(queue  )  


def to_postfix(regex):
    if(validate_regex(regex)):
       regex = handle_dot(regex)
       print(regex)
       regex = handle_range(regex)
       print(regex)
       regex = concatenate(regex)
       print(regex)
       regex = infix_to_postfix(regex)
       print(regex)

       return regex


    else:
        print('sorry you have entered invalid regex! ')    


    

# print(handle_dot('ab.c'))
# print(handle_range("[a-z]+abc[Abc]*[1-5]"))
# print(concatenate("a+b*|c(5.f)"))
# print(infix_to_postfix("(A+.B*)?.(C|D)"))
print(to_postfix("."))        


(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9)
(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9)
(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9)
ab|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9|
ab|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9|


# Postfix to NFA

In [84]:
class edge:
    def __init__(self):
       self.label=None
       self.destination=None


class state:
       def __init__(self):
        self.label=None
        self.outgoing_edges=[]




class NFA:    
       def __init__(self, start, accept,states):
        self.start_state = start
        self.acceptance_state = accept
        self.states = states
        

def construct_nfa (label, id, stack):
    start = state()
    acceptance = state()
    start.label = "S"+str(id)
    acceptance.label = "S"+str(id+1)

    Edge = edge()
    Edge.label = label
    Edge.destination = acceptance 
    start.outgoing_edges.append(Edge)

    result_nfa = NFA(start, acceptance, [start,acceptance])
    stack.append(result_nfa)
    #print(result_nfa.inner_states)

    return result_nfa, id+2


def nfa_concatination (stack):
   
    
    nfa2 = stack.pop()
    nfa1 = stack.pop()


    Edge = edge()
    Edge.label = 'ε'
    Edge.destination = nfa2.start_state
    nfa1.acceptance_state.outgoing_edges.append(Edge)

    #print(nfa1)
    #print(nfa1.inner_states)

    resultnfa = NFA(nfa1.start_state,  nfa2.acceptance_state, nfa1.states + nfa2.states)
    stack.append(resultnfa)
    return resultnfa


def nfa_or (stack, id):
  nfa1 = stack.pop()
  nfa2 = stack.pop()
  newStart = state()
  newStart.label= "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  Edge1 = edge()
  Edge1.label = 'ε'
  Edge1.destination = nfa1.start_state

  Edge2 = edge()
  Edge2.label = 'ε'
  Edge2.destination = nfa2.start_state

  newStart.outgoing_edges.append(Edge1)
  newStart.outgoing_edges.append(Edge2)


  Edge3 = edge()
  Edge3.label = 'ε'
  Edge3.destination = newEnd

  nfa1.acceptance_state.outgoing_edges.append(Edge3)

  Edge4 = edge()
  Edge4.label = 'ε'
  Edge4.destination = newEnd
  nfa2.acceptance_state.outgoing_edges.append(Edge4)


  result = NFA(newStart, newEnd, [newStart,newEnd]+ nfa1.states + nfa2.states)
  stack.append(result)
  return result, id+2



    
def nfa_json (nefa:NFA):
  outputJson = dict()
  outputJson["startingState"] = nefa.start_state.label
  for stat in nefa.states:
    stateDict = dict()
    if stat == nefa.acceptance_state:
      stateDict["isTerminatingState"] = True
    else:
      stateDict["isTerminatingState"] = False
    for edg in stat.outgoing_edges:
      if(edg.label in stateDict.keys()):
        stateDict[edg.label] = [] + [stateDict[edg.label]] + [edg.destination.label]
      else:
        stateDict[edg.label] = edg.destination.label
    outputJson[stat.label] = stateDict
  nfaOutFile = open('NFA.json', 'w')
  JsonObject = json.dump(outputJson, nfaOutFile,indent=6, ensure_ascii=False)
  nfaOutFile.close()
  return JsonObject

def postfix_to_nfa(postfix):
     stack = []
     id = 1

     for i in range(len(postfix)):
         if postfix[i] in alphanumeric:
           _ ,id =  construct_nfa(postfix[i],id,stack)
         elif postfix[i] == '.':   
             nfa_concatination(stack) 
         elif postfix[i] == '|':   
            _,id = nfa_or(stack,id)     
     result = stack.pop()
     return result







In [88]:
postfix = to_postfix("a[ab]|b")
nfa = postfix_to_nfa(postfix)
json_obj = nfa_json(nfa)


a[ab]|b
a(a|b)|b
a.(a|b)|b
aab|.b|


# NFA to DFA

# Minimized DFA